In [1]:
require 'open-uri'
require 'json'
require 'daru'
require 'gsl'
require 'distribution'
require 'sqlite3' ##This should return true if you have SQLite3 installed
require './regression'
include Regression

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Object

In [2]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [3]:
begin db.execute("drop table cross_validation"); rescue; end
sql = <<SQL
create temp table cross_validation as 
  select sk_id_curr, abs(random()) % 10 as fold
  from application_train
  order by random()
SQL
db.execute(sql)

[]

In [4]:
sql = <<SQL
create temp view example_regression as 
  select a.sk_id_curr, 
  target,
  fold, ext_source_1, ext_source_2, ext_source_3
  from application_train a inner join cross_validation c on a.sk_id_curr = c.sk_id_curr
SQL
begin db.execute("drop view example_regression"); rescue; end
db.execute(sql)

[]

In [5]:
column_types = Hash.new
db.execute("PRAGMA table_info(application_train);").each do |row|
  column_types[row["name"]] = row["type"]
end
1

1

In [27]:
def plot x, y
  Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "X"
    plot.y_label "Y"
  end
end

:plot

In [12]:
def roc_curve(scores)
  totals = Hash.new {|h,k| h[k] = 0.0} 
  scores.each do |j,k|
    totals[k] += 1.0
  end
  
  computed = Hash.new{|h,k| h[k]=0.0}
  false_positive = [0.0]
  true_positive = [0.0]
  auc = 0.0
  
  scores.sort_by{ |scor,pred| -scor}.each do |score,predicted|
    computed[predicted] += 1
    fpr = computed[0]/totals[0]
    tpr = computed[1]/totals[1]
    auc += 0.5*(fpr-false_positive.last)*(tpr+true_positive.last)
    false_positive << fpr
    true_positive << tpr
  end
  return[false_positive,true_positive,auc]
end

:roc_curve

## Fixed Classifier

In [24]:
class FixedClassifier 
  def predict data, w
    return 1.0
  end
end

:predict

In [36]:
batch_data = db.execute("SELECT SK_ID_CURR, TARGET, CNT_CHILDREN, FLAG_MOBIL, FLAG_EMP_PHONE, FLAG_WORK_PHONE, FLAG_CONT_MOBILE, FLAG_PHONE, FLAG_EMAIL, CNT_FAM_MEMBERS, REGION_RATING_CLIENT, REGION_RATING_CLIENT_W_CITY, HOUR_APPR_PROCESS_START, REG_REGION_NOT_LIVE_REGION, REG_REGION_NOT_WORK_REGION, LIVE_REGION_NOT_WORK_REGION, REG_CITY_NOT_LIVE_CITY, REG_CITY_NOT_WORK_CITY, LIVE_CITY_NOT_WORK_CITY, NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, EMERGENCYSTATE_MODE, OBS_30_CNT_SOCIAL_CIRCLE, DEF_30_CNT_SOCIAL_CIRCLE, OBS_60_CNT_SOCIAL_CIRCLE, DEF_60_CNT_SOCIAL_CIRCLE FROM application_train limit 50000;")
data = labelData batch_data , column_types
1

1

In [33]:
sql = "SELECT SK_ID_CURR, TARGET, LIVE_CITY_NOT_WORK_CITY FROM application_train limit 50000;"
scores = score_model FixedClassifier.new, db, nil, sql
f_p, t_p, auc = roc_curve scores
puts auc
plot f_p, t_p

0.49626408745123196


#<Nyaplot::Plot:0x007fce2d140f20 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fce0fdb2808 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"f5495026-6d0d-4dde-b279-796327c3c80b"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

### Fixed Classifier
*Accuracy is 0.49626408745123196*
![classifier](fixed_roc.png)


## Random Classifier

In [52]:
class RandomClasifier 
  
  def func dataset
    count = 0.0
    total = 0.0
    dataset.each{|row|
      if row["label"] == 0
        count += 1
      end
      total += 1
      }
    w = [count/total]
    return w
  end
  
  def predict data, w
    pred = rand
    if rand < pred
      return pred
    else
      return pred
    end
  end
end

:predict

In [53]:
rgc = RandomClasifier.new
w = rgc.func data

[0.91948]

In [55]:
sql = "SELECT SK_ID_CURR, TARGET, EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 FROM application_train limit 5000;"
scores = score_model rgc, db, w, sql
# puts scores
f_p, t_p, auc = roc_curve scores
puts auc
plot f_p, t_p

0.5108079487749598


#<Nyaplot::Plot:0x007fcdfd1bb4a8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fcdfbd38210 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"e1916718-1c61-4feb-a945-0745d63859ca"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

### Random Classifier
*Accuracy is 0.5108079487749598*
![classifier](Random_roc.png)